In [15]:
import pandas as pd
from difflib import *
import string
import re
import unicodedata
from difflib import get_close_matches


# Rodrigo deu essa ideia de boas práticas na limpesa dele e só aproveitei
PATH = 'seeders/olist_geolocation_dataset.csv'
CLEAN_PATH = 'clean_datasets/geolocation.csv'

# Deixei como df para padronizar os arquivos
df = pd.read_csv(PATH)

# As duas colunas que eu não encontrei uma necessidade de existência, e que podem ser retiradas para melhorar o trabalho, são de latitude e longitude
# O zip code é 'um CEP americano', preferi manter, pois a informação pode ser útil no calculo de rotas

# removendo as colunas de latitude e longitude
df = df.drop('geolocation_lat', axis=1)
df = df.drop('geolocation_lng', axis=1)


# Nomeação das colunas
df.columns = ['zip_code_prefixo', 'cidade', 'sigla_uf']
df.shape
df.dtypes
df.shape

# existem problemas de cidades com o mesmo nome, e formas diferentes de escrita. Como solucionar de forma prática? Sem verificar cidade por cidade
# Primeira regulamentação: Deixando todas as cidades completamente minúsculas
df['cidade'] = df['cidade'].str.lower()

# Criar um dicionário para armazenar as cidades normalizadas separadas por inicial
cidades_normalizadas = {}

# Criar uma função para normalizar cidades
# Apesar de demorar mais de 10 minutos, foi a forma mais rápida. As outras passaram facilmente de 20 minutos de execução
def normalizar_cidade(cidade):
    inicial = cidade[0].lower()  # Obter a inicial da cidade em minúsculas
    cidades_inicial = cidades_normalizadas.get(inicial, [])  # Obter a lista de cidades para essa inicial
    cidade_normalizada = get_close_matches(cidade, cidades_inicial, n=1, cutoff=0.8)  # Buscar cidade normalizada
    if cidade_normalizada:
        return cidade_normalizada[0]
    else:
        cidades_inicial.append(cidade)  # Adicionar cidade à lista para essa inicial
        cidades_normalizadas[inicial] = cidades_inicial  # Atualizar dicionário com a nova lista de cidades
        return cidade


# Salvar o DataFrame de volta ao arquivo CSV, se desejar
# df.to_csv('seu_arquivo_normalizado.csv', index=False)

df['cidade'] = df['cidade'].apply(normalizar_cidade)

cidades = df['cidade'].unique()


# Alguns valores ficaram com acentos, vamos removê-los

def remover_acentos(texto):
    # Normaliza o texto para NFC (Formulário Normalizado de Composição)
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    # Remove os caracteres que não são ASCII mantendo as letras intactas
    texto_sem_acentos = re.sub(r'[^\x00-\x7F]', '', texto_normalizado)
    return texto_sem_acentos

df['cidade'] = df['cidade'].apply(remover_acentos)

cidades = df['cidade'].unique()
cidades

mapeamento_regioes = {
    'Norte': ['AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO'],
    'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    'Centro-Oeste': ['DF', 'GO', 'MS', 'MT'],
    'Sudeste': ['ES', 'MG', 'RJ', 'SP'],
    'Sul': ['PR', 'RS', 'SC']
}

def mapear_regiao(sigla):
    for regiao, estados in mapeamento_regioes.items():
        if sigla in estados:
            return regiao
    return 'Outros'

df['regiao'] = df['sigla_uf'].apply(mapear_regiao)

df = df.drop('Região', axis=1)

# Cria um novo arquivo csv já limpo e salva nesse caminho
df.to_csv(CLEAN_PATH, index=False)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


,zip_code_prefixo,cidade,sigla_uf
0,1037,sao paulo,SP
1,1046,sao paulo,SP
2,1046,sao paulo,SP
3,1041,sao paulo,SP
4,1035,sao paulo,SP
...,...,...,...
1000158,99950,tapejara,RS
1000159,99900,getulio vargas,RS
1000160,99950,tapejara,RS
1000161,99980,david canabarro,RS


,zip_code_prefixo,cidade,sigla_uf,regiao
0,1037,sao paulo,SP,Sudeste
1,1046,sao paulo,SP,Sudeste
2,1046,sao paulo,SP,Sudeste
3,1041,sao paulo,SP,Sudeste
4,1035,sao paulo,SP,Sudeste
...,...,...,...,...
1000158,99950,taperoa,RS,Sul
1000159,99900,getulio vargas,RS,Sul
1000160,99950,taperoa,RS,Sul
1000161,99980,david canabarro,RS,Sul


RESULTADO FINAL

CONTEXTO DA TABELA

contém detalhes da região, pode ser usado para calcular a distância entre o vendedor e o comprador

COLUNAS

zip_code_prefixo - prefixo dos primeiros 5 digitos do ZIP CODE (é como um CEP americano)
cidade, sigla_uf e regiao - nome autoexplicativo